In [ ]:
from pathlib import Path
BASE_DIR = Path("./processed_data")


In [ ]:
MODEL_NAME = 'eval'  # used for artifact filenames

In [ ]:
# Imports
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from model import HybridModel
model = HybridModel()
model.load_state_dict(torch.load('best_hybrid.pth'))
model.eval()

ModuleNotFoundError: No module named 'model'

In [ ]:
# Load data
# If needed: reload test set (X_test_l, X_test_c, y_test), rewrap into test_loader

# Run inference
test_preds, test_true = [], []
with torch.no_grad():
    for x_lstm, x_cnn, yb in test_loader:
        x_lstm, x_cnn = x_lstm.to(device), x_cnn.to(device)
        logits = model(x_lstm, x_cnn)
        preds = logits.argmax(dim=1).cpu().numpy()
        test_preds.extend(preds)
        test_true.extend(yb.numpy())


In [ ]:
# Evaluation
y_true_names = le.inverse_transform(test_true)
y_pred_names = le.inverse_transform(test_preds)

print(classification_report(y_true_names, y_pred_names, digits=4))

cm = confusion_matrix(y_true_names, y_pred_names, labels=le.classes_)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:

import json, os
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np

def _safe_metrics_dump(model_name, y_true_names, y_pred_names, labels):
    acc = accuracy_score(y_true_names, y_pred_names)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_true_names, y_pred_names, average='macro', zero_division=0)
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_true_names, y_pred_names, average='weighted', zero_division=0)
    report = classification_report(y_true_names, y_pred_names, digits=4, output_dict=True)
    outdir = Path('/mnt/data/artifacts')
    outdir.mkdir(parents=True, exist_ok=True)
    with open(outdir / f'metrics_{model_name}.json', 'w') as f:
        json.dump({
            "model": model_name,
            "accuracy": acc,
            "precision_macro": precision_macro,
            "recall_macro": recall_macro,
            "f1_macro": f1_macro,
            "precision_weighted": precision_weighted,
            "recall_weighted": recall_weighted,
            "f1_weighted": f1_weighted,
            "classification_report": report,
            "labels": list(labels)
        }, f, indent=2)
    cm = confusion_matrix(y_true_names, y_pred_names, labels=labels)
    plt.figure()
    im = plt.imshow(cm)
    plt.colorbar(im)
    plt.xticks(range(len(labels)), labels, rotation=45, ha='right')
    plt.yticks(range(len(labels)), labels)
    for (i, j), val in np.ndenumerate(cm):
        plt.text(j, i, int(val), ha='center', va='center')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix - {model_name}")
    plt.tight_layout()
    plt.savefig(outdir / f'confusion_{model_name}.png', dpi=150)
    plt.close()

try:
    _labels = le.classes_ if 'le' in globals() else sorted(set(y_true_names) | set(y_pred_names))
    _model_name = MODEL_NAME if 'MODEL_NAME' in globals() else 'model'
    _safe_metrics_dump(_model_name, y_true_names, y_pred_names, _labels)
    print(f"[patch] Saved metrics and confusion matrix for {_model_name} to /mnt/data/artifacts")
except Exception as e:
    print("[patch] Unable to compute metrics from this notebook:", e)
